## 1. Load the dataset



To run this notebook, you will need to install: pandas, openai, transformers, plotly, matplotlib, scikit-learn, torch (transformer dep), torchvision, and scipy.

In [5]:
#install openai (do this only once at the start. Source: https://community.openai.com/t/no-module-named-openai/8303/3)
!python --version
!pip --version
!pip install openai

Python 3.9.12
pip 21.2.4 from C:\Users\larsk\miniconda3\lib\site-packages\pip (python 3.9)



Import various library's

In [6]:
import pandas as pd
import time
import openai
import json
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

Create embeddings, make sure you change the API key. Currently limits to 200 rows because of token prices.

In [8]:
# Setup openAI and constants
openai.api_key = ""

# OpenAI model for embedding complaints: text-embedding-ada-002. link: https://platform.openai.com/docs/models/embeddings
embedding_model = "text-embedding-ada-002"

# Retrieve the data from the database
input_datapath = pd.read_csv('StaterData.csv')

# Limit test size due to performance issues
data = input_datapath.loc[:20]

# Create and return embedding
def get_embedding(text, model="text-embedding-ada-002"):
    # sleep 1 second to prevent reaching rate limit. Limit: 60 requests per min.
    time.sleep(1)
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

data['Embedding'] = data['Clean consumer complaint'].apply(lambda x: get_embedding(x))
data.to_csv("StaterDataEmbeddings.csv", index=False)

C:\Users\larsk\AppData\Local\Temp\ipykernel_16372\2179164496.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Embedding'] = data['Clean consumer complaint'].apply(lambda x: get_embedding(x))


Print head of data to check if the data is correct

In [11]:
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Complaint ID                  21 non-null     int64 
 1   Issue                         21 non-null     object
 2   Consumer complaint narrative  21 non-null     object
 3   Clean consumer complaint      21 non-null     object
 4   Embedding                     21 non-null     object
dtypes: int64(1), object(4)
memory usage: 968.0+ bytes


Used random forest to calculate the accuracy of the embeddings using OpenAI

In [10]:
# Prepare the feature matrix X and target vector y
X = data["Embedding"].apply(lambda x: json.loads(x)).tolist()
y = data["Issue"]

# Reshape the embeddings into a 2D array
X = np.array(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Create an instance of the random forest classifier
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=2)

# Train the random forest classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the performance of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

TypeError: the JSON object must be str, bytes or bytearray, not list

Accuracy for this model is 0.54. It is an ok model as the complaints are not cleaned yet.